Dataset link:

https://www.kaggle.com/datasets/ramisaalam/squad-translated-to-hindi

#Initialising the project

Installing required libraries and importing them.

In [ ]:
!pip install simpletransformers

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

Dataset is a csv file, that I have loaded on my drive.

In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/NLPDataset/Dataset.csv', encoding='utf-8-sig')

#Data Exploration

In [ ]:
df.head()

,context,question,answer_start,answer_text,language
0,"वास्तुशिल्प रूप से, स्कूल में एक कैथोलिक चरित्...",नोट्रे डेम में ग्रोटो क्या है?,332,प्रार्थना और प्रतिबिंब का एक मैरियन स्थान,hindi
1,"वास्तुशिल्प रूप से, स्कूल में एक कैथोलिक चरित्...",नोट्रे डेम में मुख्य भवन के शीर्ष पर क्या बैठत...,82,वर्जिन मैरी की एक सुनहरी मूर्ति,hindi
2,"अधिकांश अन्य विश्वविद्यालयों के रूप में, नोट्र...",नोट्रे डेम की विद्वान पत्रिका कब प्रकाशित हुई?,211,सितंबर 1876,hindi
3,"अधिकांश अन्य विश्वविद्यालयों के रूप में, नोट्र...",कितनी बार नोट्रे डेम का जुगलर प्रकाशित होता है?,429,दो बार,hindi
4,"अधिकांश अन्य विश्वविद्यालयों के रूप में, नोट्र...",नोट्रे डेम में कितने छात्र समाचार पत्र पाए जात...,124,तीन,hindi


In [ ]:
df.shape

(4734, 5)

The format of Question answering system:
Context, Question and the Answer.

In [ ]:
print(f"Context:\n{df['context'][0]}\n\nQuestion:\n{df['question'][0]}\n\nAnswer:\n{df['answer_text'][0]}")

Context:
वास्तुशिल्प रूप से, स्कूल में एक कैथोलिक चरित्र है। मुख्य भवन के गोल्ड डोम के ऊपर वर्जिन मैरी की एक सुनहरी मूर्ति है। मुख्य भवन के सामने और इसका सामना करना पड़ता है, क्राइस्ट के साथ एक तांबा मूर्ति है जो किंवदंती "वेनिट एडी मी ओमनेस" के साथ उगता है। मुख्य भवन के बगल में पवित्र हृदय का बेसिलिका है। बेसिलिका के पीछे तुरंत ग्रोट्टो, प्रार्थना और प्रतिबिंब का एक मैरियन स्थान है। यह लॉर्ड्स, फ्रांस में ग्रोट्टो की एक प्रतिकृति है जहां वर्जिन मैरी ने 1858 में सेंट बर्नडेट सॉबिरस को सम्मानित किया था। मुख्य ड्राइव के अंत में (और एक सीधी रेखा में जो 3 मूर्तियों और सोने के गुंबद से जुड़ता है), मैरी की एक सरल, आधुनिक पत्थर की मूर्ति है।

Question:
नोट्रे डेम में ग्रोटो क्या है?

Answer:
प्रार्थना और प्रतिबिंब का एक मैरियन स्थान


In [ ]:
df['context'][0]

'वास्तुशिल्प रूप से, स्कूल में एक कैथोलिक चरित्र है। मुख्य भवन के गोल्ड डोम के ऊपर वर्जिन मैरी की एक सुनहरी मूर्ति है। मुख्य भवन के सामने और इसका सामना करना पड़ता है, क्राइस्ट के साथ एक तांबा मूर्ति है जो किंवदंती "वेनिट एडी मी ओमनेस" के साथ उगता है। मुख्य भवन के बगल में पवित्र हृदय का बेसिलिका है। बेसिलिका के पीछे तुरंत ग्रोट्टो, प्रार्थना और प्रतिबिंब का एक मैरियन स्थान है। यह लॉर्ड्स, फ्रांस में ग्रोट्टो की एक प्रतिकृति है जहां वर्जिन मैरी ने 1858 में सेंट बर्नडेट सॉबिरस को सम्मानित किया था। मुख्य ड्राइव के अंत में (और एक सीधी रेखा में जो 3 मूर्तियों और सोने के गुंबद से जुड़ता है), मैरी की एक सरल, आधुनिक पत्थर की मूर्ति है।'

In [ ]:
df['question'][0]

'नोट्रे डेम में ग्रोटो क्या है?'

In [ ]:
df['answer_text'][0]

'प्रार्थना और प्रतिबिंब का एक मैरियन स्थान'

Number of unique contexts:

In [ ]:
df['context'].nunique()

1728

In [ ]:
df.dtypes

context         object
question        object
answer_start     int64
answer_text     object
language        object
dtype: object

#Data Preprocessing

Checking if all the instances are in Hindi language.

In [ ]:
df['language'].unique()

array(['hindi'], dtype=object)

Checking if the value in "answer_start" actually corresponds to our answer:

In [ ]:
df.head()

,context,question,answer_start,answer_text,language
0,"वास्तुशिल्प रूप से, स्कूल में एक कैथोलिक चरित्...",नोट्रे डेम में ग्रोटो क्या है?,332,प्रार्थना और प्रतिबिंब का एक मैरियन स्थान,hindi
1,"वास्तुशिल्प रूप से, स्कूल में एक कैथोलिक चरित्...",नोट्रे डेम में मुख्य भवन के शीर्ष पर क्या बैठत...,82,वर्जिन मैरी की एक सुनहरी मूर्ति,hindi
2,"अधिकांश अन्य विश्वविद्यालयों के रूप में, नोट्र...",नोट्रे डेम की विद्वान पत्रिका कब प्रकाशित हुई?,211,सितंबर 1876,hindi
3,"अधिकांश अन्य विश्वविद्यालयों के रूप में, नोट्र...",कितनी बार नोट्रे डेम का जुगलर प्रकाशित होता है?,429,दो बार,hindi
4,"अधिकांश अन्य विश्वविद्यालयों के रूप में, नोट्र...",नोट्रे डेम में कितने छात्र समाचार पत्र पाए जात...,124,तीन,hindi


In [ ]:
df['new'] = df.apply(lambda x: x['context'][x['answer_start']:], axis = 1)

In [ ]:
df.head()

,context,question,answer_start,answer_text,language,new
0,"वास्तुशिल्प रूप से, स्कूल में एक कैथोलिक चरित्...",नोट्रे डेम में ग्रोटो क्या है?,332,प्रार्थना और प्रतिबिंब का एक मैरियन स्थान,hindi,प्रार्थना और प्रतिबिंब का एक मैरियन स्थान है। ...
1,"वास्तुशिल्प रूप से, स्कूल में एक कैथोलिक चरित्...",नोट्रे डेम में मुख्य भवन के शीर्ष पर क्या बैठत...,82,वर्जिन मैरी की एक सुनहरी मूर्ति,hindi,वर्जिन मैरी की एक सुनहरी मूर्ति है। मुख्य भवन ...
2,"अधिकांश अन्य विश्वविद्यालयों के रूप में, नोट्र...",नोट्रे डेम की विद्वान पत्रिका कब प्रकाशित हुई?,211,सितंबर 1876,hindi,सितंबर 1876 में एक-पेज जर्नल के रूप में शुरू ह...
3,"अधिकांश अन्य विश्वविद्यालयों के रूप में, नोट्र...",कितनी बार नोट्रे डेम का जुगलर प्रकाशित होता है?,429,दो बार,hindi,दो बार जारी की जाती है और छात्र साहित्य और कला...
4,"अधिकांश अन्य विश्वविद्यालयों के रूप में, नोट्र...",नोट्रे डेम में कितने छात्र समाचार पत्र पाए जात...,124,तीन,hindi,"तीन समाचार पत्र, एक रेडियो और टेलीविजन स्टेशन,..."


In [ ]:
df["val"] = df.apply(lambda y: y['new'].split(' ')[0] == y['answer_text'].split(' ')[0], axis=1)

In [ ]:
df.head()

,context,question,answer_start,answer_text,language,new,val
0,"वास्तुशिल्प रूप से, स्कूल में एक कैथोलिक चरित्...",नोट्रे डेम में ग्रोटो क्या है?,332,प्रार्थना और प्रतिबिंब का एक मैरियन स्थान,hindi,प्रार्थना और प्रतिबिंब का एक मैरियन स्थान है। ...,True
1,"वास्तुशिल्प रूप से, स्कूल में एक कैथोलिक चरित्...",नोट्रे डेम में मुख्य भवन के शीर्ष पर क्या बैठत...,82,वर्जिन मैरी की एक सुनहरी मूर्ति,hindi,वर्जिन मैरी की एक सुनहरी मूर्ति है। मुख्य भवन ...,True
2,"अधिकांश अन्य विश्वविद्यालयों के रूप में, नोट्र...",नोट्रे डेम की विद्वान पत्रिका कब प्रकाशित हुई?,211,सितंबर 1876,hindi,सितंबर 1876 में एक-पेज जर्नल के रूप में शुरू ह...,True
3,"अधिकांश अन्य विश्वविद्यालयों के रूप में, नोट्र...",कितनी बार नोट्रे डेम का जुगलर प्रकाशित होता है?,429,दो बार,hindi,दो बार जारी की जाती है और छात्र साहित्य और कला...,True
4,"अधिकांश अन्य विश्वविद्यालयों के रूप में, नोट्र...",नोट्रे डेम में कितने छात्र समाचार पत्र पाए जात...,124,तीन,hindi,"तीन समाचार पत्र, एक रेडियो और टेलीविजन स्टेशन,...",True


In [ ]:
df["val"].value_counts()

val
True     3637
False    1097
Name: count, dtype: int64

...there are 1097 instances where the value of "answer_start" doesn't match the actual start of the answer!

Let's solve this problem by finding the answer text in the context and update the answer_start...

In [ ]:
df['ans_start'] = df.apply(lambda x : x['context'].find(x['answer_text']), axis=1)

In [ ]:
df.head(10)

,context,question,answer_start,answer_text,language,new,val,ans_start
0,"वास्तुशिल्प रूप से, स्कूल में एक कैथोलिक चरित्...",नोट्रे डेम में ग्रोटो क्या है?,332,प्रार्थना और प्रतिबिंब का एक मैरियन स्थान,hindi,प्रार्थना और प्रतिबिंब का एक मैरियन स्थान है। ...,True,332
1,"वास्तुशिल्प रूप से, स्कूल में एक कैथोलिक चरित्...",नोट्रे डेम में मुख्य भवन के शीर्ष पर क्या बैठत...,82,वर्जिन मैरी की एक सुनहरी मूर्ति,hindi,वर्जिन मैरी की एक सुनहरी मूर्ति है। मुख्य भवन ...,True,82
2,"अधिकांश अन्य विश्वविद्यालयों के रूप में, नोट्र...",नोट्रे डेम की विद्वान पत्रिका कब प्रकाशित हुई?,211,सितंबर 1876,hindi,सितंबर 1876 में एक-पेज जर्नल के रूप में शुरू ह...,True,211
3,"अधिकांश अन्य विश्वविद्यालयों के रूप में, नोट्र...",कितनी बार नोट्रे डेम का जुगलर प्रकाशित होता है?,429,दो बार,hindi,दो बार जारी की जाती है और छात्र साहित्य और कला...,True,277
4,"अधिकांश अन्य विश्वविद्यालयों के रूप में, नोट्र...",नोट्रे डेम में कितने छात्र समाचार पत्र पाए जात...,124,तीन,hindi,"तीन समाचार पत्र, एक रेडियो और टेलीविजन स्टेशन,...",True,124
5,विश्वविद्यालय पवित्र क्रॉस की मंडली की प्रमुख ...,जहां पवित्र क्रॉस की मण्डली का मुख्यालय है?,93,रोम,hindi,"रोम में हैं)। इसका मुख्य सेमिनरी, मोरौ सेमिनरी...",True,93
6,विश्वविद्यालय पवित्र क्रॉस की मंडली की प्रमुख ...,पवित्र क्रॉस की मण्डली का प्राथमिक मित्रता क्य...,127,मोरौ सेमिनरी,hindi,"मोरौ सेमिनरी, सेंट पर कैंपस पर स्थित है। मुख्य...",True,127
7,विश्वविद्यालय पवित्र क्रॉस की मंडली की प्रमुख ...,क्या व्यक्ति नोट्रे डेम में फातिमा हाउस में रह...,305,सेवानिवृत्त पुजारी और भाई,hindi,सेवानिवृत्त पुजारी और भाई फातिमा हाउस (एक पूर्...,True,305
8,कॉलेज ऑफ इंजीनियरिंग की स्थापना 1920 में हुई थ...,नोट्रे डेम में इंजीनियरिंग कॉलेज में कितने बीए...,451,आठ,hindi,"बी के साथ। डिग्री की पेशकश की। इसके अतिरिक्त,...",False,449
9,कॉलेज ऑफ इंजीनियरिंग की स्थापना 1920 में हुई थ...,कॉलेज ऑफ इंजीनियरिंग के समान अध्ययन के निर्माण...,128,कॉलेज ऑफ साइंस,hindi,"लेज ऑफ साइंस का हिस्सा थे। आज कॉलेज, फिट्जपैट्...",False,126


Done!
Now removing the unnecessary columns...

In [ ]:
df = df.drop(columns=['answer_start', 'new', 'val', 'language'])

In [ ]:
df.shape

(4734, 4)

This is the final dataframe...

In [ ]:
df.head()

,context,question,answer_text,ans_start
0,"वास्तुशिल्प रूप से, स्कूल में एक कैथोलिक चरित्...",नोट्रे डेम में ग्रोटो क्या है?,प्रार्थना और प्रतिबिंब का एक मैरियन स्थान,332
1,"वास्तुशिल्प रूप से, स्कूल में एक कैथोलिक चरित्...",नोट्रे डेम में मुख्य भवन के शीर्ष पर क्या बैठत...,वर्जिन मैरी की एक सुनहरी मूर्ति,82
2,"अधिकांश अन्य विश्वविद्यालयों के रूप में, नोट्र...",नोट्रे डेम की विद्वान पत्रिका कब प्रकाशित हुई?,सितंबर 1876,211
3,"अधिकांश अन्य विश्वविद्यालयों के रूप में, नोट्र...",कितनी बार नोट्रे डेम का जुगलर प्रकाशित होता है?,दो बार,277
4,"अधिकांश अन्य विश्वविद्यालयों के रूप में, नोट्र...",नोट्रे डेम में कितने छात्र समाचार पत्र पाए जात...,तीन,124


#Data conversion

Converting our dataframe to the following format:

In [ ]:
# {
#   context: “sample context”,
#   qas: [{
#     id:1,
#     is_impossible: False,
#     question: “sample question”,
#     answers:[{text: “answer text”,
#     answer_start: 332}]}]
# }

...in order to use the QuestionAnswering model

In [ ]:
context = df['context'][0]

In [ ]:
data = []
qas = []

for i in range(4734):
  if context == df['context'][i]:
    question_text = df['question'][i]
    answer_text = df['answer_text'][i]

    ans_start = df['ans_start'][i]

    is_impossible = False

    if ans_start == -1:
      is_impossible = True
      qas.append({
          'id' : str(i + 1),
          'is_impossible' : is_impossible,
          'question' : question_text,
          'answers' : []
      })
    else :
      answer = []
      answer.append({
          'text' : answer_text,
          'answer_start' : ans_start
      })

      qas.append({
          'id' : str(i + 1),
          'is_impossible' : is_impossible,
          'question' : question_text,
          'answers' : answer
      })
  else:
    data.append({
        'context' : context,
        'qas' : qas
    })

    context = df['context'][i]
    qas = []
    question_text = df['question'][i]
    answer_text = df['answer_text'][i]

    ans_start = df['ans_start'][i]

    is_impossible = False

    if ans_start == -1:
      is_impossible = True
      qas.append({
          'id' : str(i + 1),
          'is_impossible' : is_impossible,
          'question' : question_text,
          'answers' : []
      })
    else :
      answer = []
      answer.append({
          'text' : answer_text,
          'answer_start' : ans_start
      })

      qas.append({
          'id' : str(i + 1),
          'is_impossible' : is_impossible,
          'question' : question_text,
          'answers' : answer
      })


data.append({
    'context' : context,
    'qas' : qas
})


In [ ]:
data[0]

{'context': 'वास्तुशिल्प रूप से, स्कूल में एक कैथोलिक चरित्र है। मुख्य भवन के गोल्ड डोम के ऊपर वर्जिन मैरी की एक सुनहरी मूर्ति है। मुख्य भवन के सामने और इसका सामना करना पड़ता है, क्राइस्ट के साथ एक तांबा मूर्ति है जो किंवदंती "वेनिट एडी मी ओमनेस" के साथ उगता है। मुख्य भवन के बगल में पवित्र हृदय का बेसिलिका है। बेसिलिका के पीछे तुरंत ग्रोट्टो, प्रार्थना और प्रतिबिंब का एक मैरियन स्थान है। यह लॉर्ड्स, फ्रांस में ग्रोट्टो की एक प्रतिकृति है जहां वर्जिन मैरी ने 1858 में सेंट बर्नडेट सॉबिरस को सम्मानित किया था। मुख्य ड्राइव के अंत में (और एक सीधी रेखा में जो 3 मूर्तियों और सोने के गुंबद से जुड़ता है), मैरी की एक सरल, आधुनिक पत्थर की मूर्ति है।',
 'qas': [{'id': '1',
   'is_impossible': False,
   'question': 'नोट्रे डेम में ग्रोटो क्या है?',
   'answers': [{'text': 'प्रार्थना और प्रतिबिंब का एक मैरियन स्थान',
     'answer_start': 332}]},
  {'id': '2',
   'is_impossible': False,
   'question': 'नोट्रे डेम में मुख्य भवन के शीर्ष पर क्या बैठता है?',
   'answers': [{'text': 'वर्जिन मैरी की एक सु

#Train-test split of data

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.1)

#Instantiating Models

Model 1  is IndicBERT.

In [ ]:
train_args_1 = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"output/albert",
    "best_model_dir": f"output/albert/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 512,
    "num_train_epochs": 10,
    "evaluate_during_training_steps": 2500,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": 'ai4bharat/indic-bert'},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size": 3,
    "train_batch_size": 32,
    "eval_batch_size": 32
    #"lazy_loading": True
}

In [ ]:
model_1 = QuestionAnsweringModel(
    "albert", "ai4bharat/indic-bert", args=train_args_1, use_cuda=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

Some weights of AlbertForQuestionAnswering were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

Model 2 is DistilBERT

In [ ]:
train_args_2 = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"output/roberta",
    "best_model_dir": f"output/roberta/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 512,
    "num_train_epochs": 10,
    "evaluate_during_training_steps": 2500,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": 'distilbert/distilroberta-base'},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size": 3,
    "train_batch_size": 32,
    "eval_batch_size": 32
    #"lazy_loading": True
}

In [ ]:
model_2 = QuestionAnsweringModel(
    "roberta", "distilbert/distilroberta-base", args=train_args_2, use_cuda=True
)

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

#Training Models

IndicBERT

In [ ]:
model_1.train_model(train_data, eval_data=test_data)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
add example index and unique id: 100%|██████████| 4287/4287 [00:00<00:00, 371009.62it/s]


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


...

Running Epoch 9 of 10:   0%|          | 0/137 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 10 of 10:   0%|          | 0/137 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/15 [00:00<?, ?it/s]

(1370,
 {'global_step': [137, 274, 411, 548, 685, 822, 959, 1096, 1233, 1370],
  'correct': [64, 118, 118, 118, 119, 109, 107, 103, 104, 99],
  'similar': [111, 122, 122, 123, 122, 134, 123, 116, 113, 119],
  'incorrect': [272, 207, 207, 206, 206, 204, 217, 228, 230, 229],
  'train_loss': [2.7408852577209473,
   2.9980063438415527,
   2.4887897968292236,
   0.7616899013519287,
   1.326755166053772,
   1.079903483390808,
   1.1242125034332275,
   0.961172342300415,
   0.4855397641658783,
   0.16185548901557922],
  'eval_loss': [-2.6376302083333334,
   -3.4428385416666667,
   -3.8055989583333334,
   -4.23359375,
   -4.408333333333333,
   -4.587239583333333,
   -4.75546875,
   -4.911458333333333,
   -4.98046875,
   -5.0359375]})

DistilBERT

In [ ]:
model_2.train_model(train_data, eval_data=test_data)

add example index and unique id: 100%|██████████| 4287/4287 [00:00<00:00, 231800.30it/s]


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Training loss,█▇▅▅▅▃▄▃▃▃▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁
correct,▁████▇▆▆▆▅
eval_loss,█▆▅▃▃▂▂▁▁▁
global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
incorrect,█▁▁▁▁▁▂▃▄▄
lr,▅██▇▇▇▇▆▆▆▅▅▅▅▄▄▄▄▃▃▃▂▂▂▂▁▁
similar,▁▄▄▅▄█▅▃▂▃
train_loss,▇█▇▂▄▃▃▃▂▁
Training loss,0.43941
correct,99
eval_loss,-5.03594


Running Epoch 1 of 10:   0%|          | 0/414 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 447/447 [00:07<00:00, 63.80it/s]

add example index and unique id: 100%|██████████| 447/447 [00:00<00:00, 221692.55it/s]


...

Running Epoch 9 of 10:   0%|          | 0/414 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/43 [00:00<?, ?it/s]

Running Epoch 10 of 10:   0%|          | 0/414 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/43 [00:00<?, ?it/s]

(4140,
 {'global_step': [414,
   828,
   1242,
   1656,
   2070,
   2484,
   2500,
   2898,
   3312,
   3726,
   4140],
  'correct': [7, 41, 50, 75, 61, 66, 71, 80, 80, 81, 73],
  'similar': [440, 179, 257, 161, 233, 225, 162, 154, 138, 171, 163],
  'incorrect': [0, 227, 140, 211, 153, 156, 214, 213, 229, 195, 211],
  'train_loss': [2.45013427734375,
   1.6082763671875,
   0.06961822509765625,
   0.9063873291015625,
   0.089752197265625,
   0.1136474609375,
   1.1718838214874268,
   1.32513427734375,
   0.1944732666015625,
   0.3721122741699219,
   0.69677734375],
  'eval_loss': [-5.3839026162790695,
   -5.3856286337209305,
   -6.1234556686046515,
   -6.405341569767442,
   -6.618822674418604,
   -6.771529796511628,
   -6.719567587209302,
   -6.704305959302325,
   -6.888989825581396,
   -7.115098110465116,
   -7.191587936046512]})

#Evaluation of Models

In [ ]:
to_predict = [
    {
        "context": "वास्तुशिल्प रूप से, स्कूल में एक कैथोलिक चरित्र है। मुख्य भवन के गोल्ड डोम के ऊपर वर्जिन मैरी की एक सुनहरी मूर्ति है। मुख्य भवन के सामने और इसका सामना करना पड़ता है, क्राइस्ट के साथ एक तांबा मूर्ति है जो किंवदंती 'वेनिट एडी मी ओमनेस' के साथ उगता है। मुख्य भवन के बगल में पवित्र हृदय का बेसिलिका है। बेसिलिका के पीछे तुरंत ग्रोट्टो, प्रार्थना और प्रतिबिंब का एक मैरियन स्थान है। यह लॉर्ड्स, फ्रांस में ग्रोट्टो की एक प्रतिकृति है जहां वर्जिन मैरी ने 1858 में सेंट बर्नडेट सॉबिरस को सम्मानित किया था। मुख्य ड्राइव के अंत में (और एक सीधी रेखा में जो 3 मूर्तियों और सोने के गुंबद से जुड़ता है), मैरी की एक सरल, आधुनिक पत्थर की मूर्ति है।",
        "qas": [
            {
                "question": "स्कूल में क्या है?",
                "id": "0",
            }
        ],
    }
]

IndicBERT

In [ ]:
model_1.predict(to_predict)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1038.45it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '0', 'answer': ['कैथोलिक चरित्र', 'एक कैथोलिक चरित्र']}],
 [{'id': '0', 'probability': [0.7218256435318623, 0.27720363174859775]}])

DistilBERT

In [ ]:
model_2.predict(to_predict)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3545.48it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '0', 'answer': ['', 'वास्तुशिल्प रूप से, स्कूल में एक कैथोलिक']}],
 [{'id': '0', 'probability': [0.42220868380144405, 0.3088944717461273]}])